<a href="https://colab.research.google.com/github/jiang28/Deep-Spatio-Temporal-Encoding/blob/master/TransGcnRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
PATH = '/content/drive/My Drive/'
device = 'cuda'
import torch
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [ ]:
#load data
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv

output_window = 1

def get_data():
    data = []
    features = []
    location = []
    label = []
    series = read_csv('DATA/data_census_update_school.csv', sep=',', header=None, low_memory=False, keep_default_na=False).to_numpy()

    for i in range(len(series)):
        if i == 0:
            continue

        # if '' in series[i]:
        #   continue

        line = series[i][2:-11]
        line = [float(_) if _ != '' else -1 for _ in line]
        line = [int(j) for j in line]

        data.append(line)

        f = series[i][-11:-1]
        f = [float(_) if _ != '' else -1 for _ in f]
        f = [int(_) for _ in f]

        features.append(f)

        label.append(int(float(series[i][-1])))

    data = np.asarray(data)
    features = np.asarray(features)
    label = np.asarray(label)

    data_ = []
    features_ = []
    for i in range(len(data)):
        scaler = MinMaxScaler(feature_range=(0, 1))
        x = data[i]
        x = scaler.fit_transform(np.array(x).reshape(-1, 1)).reshape(-1)
        data_.append(x)

        features_.append(scaler.fit_transform(np.array(features[i]).reshape(-1, 1)).reshape(-1))

    input_data = torch.FloatTensor(data_)
    label = torch.FloatTensor(label)
    features = torch.FloatTensor(features_)
    print(len(input_data))
    samples = int(len(input_data)*0.8)
    x_train = input_data[:samples]
    x_test = input_data[samples:]

    y_train = label[:samples]
    y_test = label[samples:]

    ftr_train = features[:samples]
    ftr_test = features[samples:]

    return x_train, y_train, ftr_train, x_test, y_test, ftr_test, features

if __name__ == '__main__':
    x_train, y_train, ftr_train, x_test, y_test, ftr_test, ftr_all = get_data()
    #print(x_train.shape)
    #print(y_train.shape)
    #print(ftr_train.shape)

    #print(x_test.shape)
    #print(y_test.shape)
    #print(ftr_test.shape)

    #print(ftr_all.shape)

    pass


7436


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [ ]:
#load data
import torch
def load_matrix():
    matrix = []
    with open('DATA/matrix.txt', 'r') as f:
        for i, num in enumerate(f):
            if i % 7436 == 0:
                if i != 0:
                    matrix.append(row)
                row = []
            row.append(int(num))
    matrix.append(row)
    adj = torch.FloatTensor(matrix)
    print('adj_matrix', adj[0])

    # samples = int(7436 * 0.7)
    # loc_train = range(samples)
    # loc_test = range(samples, 7436)
    # loc_val = loc_test

    # loc_train = torch.Tensor(loc_train)
    # loc_val = torch.Tensor(loc_val)
    # loc_test = torch.Tensor(loc_test)

    # loc_train = loc_train.unsqueeze(0)
    # loc_val = loc_val.unsqueeze(0)
    # loc_test = loc_test.unsqueeze(0)

    return adj

if __name__ == '__main__':
    adj = load_matrix()
    print(adj.shape)
    pass

adj_matrix tensor([-1.,  4.,  4.,  ...,  0.,  0.,  0.])
torch.Size([7436, 7436])


In [ ]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import argparse
import sys
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = True
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [ ]:
import torch
from torch import nn

class TransGCN(nn.Module):
    def __init__(self, input_size, feature_size=250,num_layers=1,dropout=0.1):
        super(TransGCN, self).__init__()

        self.gc1 = GCNLayer(10, 10)
        # self.gc2 = GCNLayer(5, 250)


        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)


        #for layer in self.transformer_encoder.children():
            #if isinstance(layer, nn.Linear):
                #print(layer.state_dict()['weight'])

#####################################################################################################


        #num_layers = self.transformer_encoder.num_layers
        #d_model = self.transformer_encoder.layers[0].self_attn.embed_dim
        #num_heads = self.transformer_encoder.layers[0].self_attn.num_heads
        #norm_first = self.transformer_encoder.layers[0].norm_first

        #batch_size = 32
        #seq_len = 7436


        #x = torch.randn((batch_size,seq_len,d_model))

        #attention_maps = []

        #with torch.no_grad():
            #for i in range(num_layers):
                # compute attention of layer i
                #h = x.clone()
                #if norm_first:
                    #h = self.transformer_encoder.layers[i].norm1(h)
                    #attn = self.transformer_encoder.layers[i].self_attn(h, h, h,attn_mask=self.src_mask,need_weights=True)[1]
                    #attention_maps.append(attn)
                    # forward of layer i
                    #x = self.transformer_encoder.layers[i](x,src_mask=self.src_mask)
        #print(attention_maps)
        ######################################################################################

        self.decoder = nn.Linear(feature_size,1)
        self.layer1 = nn.Linear(input_size, 200)
        self.layer2 = nn.Linear(200, 100)
        self.layer3 = nn.Linear(100, 1)

        self.last_layer = nn.Sigmoid()

    def forward(self, src, ftr_all, adj, st_idx, ed_idx):
        #gcn embedding loc information
        #print('gcn input shape', ftr_all.shape)
        gcn_output = self.gc1(ftr_all, adj)
        #print('gcn output shape', gcn_output.shape)
        gcn_output = F.relu(gcn_output)
        # gcn_tmp = F.relu(self.gc1(ftr_all, adj))
        # gcn_output = self.gc2(gcn_tmp, adj)

        #transformer embedding time series data
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        #print('before pos', src.shape)
        src = self.pos_encoder(src)

        #print('after pos', src.shape)

        hidden = self.transformer_encoder(src,self.src_mask)

        #print('after encoder', src.shape)

        gcn_output = gcn_output[st:ed]
        gcn_output = torch.transpose(gcn_output, 0, 1)

        gcn_output = torch.reshape(gcn_output, (gcn_output.shape[0], gcn_output.shape[1], 1))
        gcn_output = gcn_output.expand(gcn_output.shape[0], gcn_output.shape[1], hidden.shape[2])

        # concate gcn output to transformer output
        output = torch.cat((hidden, gcn_output), 0)

        output = self.decoder(output)

        output = torch.squeeze(output)
        output = output.transpose(0, 1)

        #MLP prediction
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)

        output = self.last_layer(output)

        #output attention matrix
        #print(self.transformer_encoder.layers)
        #save_output = SaveOutput()
        #attention_weights = self.transformer_encoder.layers[0].linear2.weight

        #in_proj_weight
        in_proj_attention_weights = self.transformer_encoder.layers[0].self_attn.in_proj_weight

        out_proj_attention_weights = self.transformer_encoder.layers[0].self_attn.out_proj.weight
        #print("out attn map shape: ", out_proj_attention_weights.shape)

        q_proj_attention_weights = self.transformer_encoder.layers[0].self_attn.q_proj_weight

        k_proj_attention_weights = self.transformer_encoder.layers[0].self_attn.k_proj_weight

        v_proj_attention_weights = self.transformer_encoder.layers[0].self_attn.v_proj_weight


        #attention_weights = self.transformer_encoder.weight
        in_proj_attn_array = in_proj_attention_weights.cpu().detach().numpy()
        out_proj_attn_array = out_proj_attention_weights.cpu().detach().numpy()
        #print("out attn map shape: ", out_proj_attn_array.shape)
        #q_proj_attn_array = q_proj_attention_weights.cpu().detach().numpy()
        #k_proj_attn_array = k_proj_attention_weights.cpu().detach().numpy()
        #v_proj_attn_array = v_proj_attention_weights.cpu().detach().numpy()
        #print(q_proj_attention_weights)

        #save numpy to csv

        import numpy
        import pandas as pd
        #numpy.savetxt("in_proj_attn_array", in_proj_attn_array, delimiter=",")
        #numpy.savetxt("out_proj_attn_array", out_proj_attn_array, delimiter=",")
        #numpy.savetxt("q_proj_attn_array", q_proj_attn_array, delimiter=",")
        #numpy.savetxt("k_proj_attn_array", k_proj_attn_array, delimiter=",")
        #numpy.savetxt("v_proj_attn_array", v_proj_attn_array, delimiter=",")

        list_of_numbers = list(range(0, 250))

        df3 = pd.DataFrame(out_proj_attn_array, columns=list_of_numbers)
        #df3.to_csv('DATA/out.csv')
        #print('df.shape: ', df.shape)

        import numpy as np
        import seaborn as sns
        import matplotlib.pylab as plt

        #plot heatmap
        #g=sns.heatmap(df3, xticklabels = list_of_numbers)
        #plt.show()
        #save_output = SaveOutput()
        #attention_weights = self.transformer_encoder.layers[-1].self_attn.register_forward_hook(save_output)
        #print(attn_array)

        return output



    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
# test
if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument('--mode', choices=['train', 'infer'],\
        default='train',help='Run mode')
    arg_parser.add_argument('--epoch', default='200', type=int)
    arg_parser.add_argument('--batch_size', default='32', type=int)
    args = arg_parser.parse_args(args=['--mode', 'train'])
    args = arg_parser.parse_args(args=['--epoch', '200'])
    args = arg_parser.parse_args(args=['--batch_size', '32'])

    X_train, y_train, FTR_train, X_test, y_test, FTR_test, FTR_all = get_data()

    X_train = X_train.to(device)
    y_train = y_train.to(device)
    FTR_train = FTR_train.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    FTR_test = FTR_test.to(device)

    ftr_all = FTR_all.to(device)

    #print(X_train.shape)
    #print(y_train.shape)
    #print(X_test.shape)
    #print(y_test.shape)
    #print(ftr_all.shape)

    adj = load_matrix()

    adj = adj.to(device)
    # loc_train = loc_train.to(device)
    # loc_val = loc_val.to(device)
    # loc_test = loc_test.to(device)

    # print(adj.shape)
    # print(loc_train.shape)
    # print(loc_val.shape)
    # print(loc_test.shape)

    model_path = PATH + 'MODEL/model_trans_gcn.pt'
    model = TransGCN(input_size = X_train.shape[1] + ftr_all.shape[1])
    model.to(device)


    loss_function = nn.MSELoss()

    optimizer = optim.SGD(model.parameters(), lr=0.02)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

    last_acc = 0.0

    if args.mode == 'train':

        for epoch in range(args.epoch):

            model.to(device)

            acc = []
            total_loss = 0

            acc_test = []

            for i in range(num_batch):
                sys.stdout.write('\r{0}/{1}'.format(i, num_batch))
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_train))

                x = X_train[st:ed]
                x = x.transpose(0, 1)
                x = torch.reshape(x, (x.shape[0], x.shape[1], 1))
                x = x.to(device)

                # ftr = FTR_train[st:ed]
                # ftr = ftr.transpose(0, 1)
                # ftr = torch.reshape(ftr, (ftr.shape[0], ftr.shape[1], 1))
                # ftr = ftr.to(device)

                label = y_train[st:ed]
                label = label.to(device)

                model.zero_grad()

                # junwei
                #print('\n input size', x.shape)
                output = model(x, ftr_all, adj, st, ed)

                predict = torch.squeeze(output)

                loss = loss_function(predict, label)

                acc.append(torch.sum(predict.gt(0.5) == label))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            for i in range(num_batch_test):
                st = i * args.batch_size
                ed = min((i+1) * args.batch_size, len(y_test))

                x_test = X_test[st:ed]
                x_test = x_test.transpose(0, 1)
                x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
                x_test = x_test.to(device)

                # ftr_test = FTR_test[st:ed]
                # ftr_test = ftr_test.transpose(0, 1)
                # ftr_test = torch.reshape(ftr_test, (ftr_test.shape[0], ftr_test.shape[1], 1))
                # ftr_test = ftr_test.to(device)

                label_test = y_test[st:ed]
                label_test = label_test.to(device)

                # output_test = model(x_test, ftr_test, loc_test, adj)
                output_test = model(x_test, ftr_all, adj, X_train.shape[0] + st, X_train.shape[0] + ed)
                predict_test = torch.squeeze(output_test)
                acc_test.append(torch.sum(predict_test.gt(0.5) == label_test))

            total_loss /= len(y_train)
            acc = sum(acc)*1.0/len(y_train)

            acc_test = sum(acc_test)*1.0/len(y_test)
            if epoch%10 == 0 or epoch == args.epoch - 1:
              print('\nEpoch: ', epoch)
              print('\nTraining set: Loss {0:.4f}. Acc {1:.4f}.\nTest set: Acc {2:.4f}.'.format(total_loss, acc, acc_test))

            scheduler.step()

            if acc_test > last_acc:
                torch.save(model.cpu(), model_path)
                last_acc = acc_test


In [ ]:
if __name__ == '__main__':
    X_train, y_train, FTR_train, X_test, y_test, FTR_test, FTR_all = get_data()

    num_batch = int(len(y_train)/args.batch_size) + 1

    num_batch_test = int(len(y_test)/args.batch_size) + 1

In [ ]:
#Transformer evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model_path = PATH + 'MODEL/model_trans_gcn.pt'
model = torch.load(model_path)
model.to(device)
acc_test = []
y_pred = []

for i in range(num_batch_test):
    st = i * args.batch_size
    ed = min((i+1) * args.batch_size, len(y_test))

    x_test = X_test[st:ed]
    x_test = x_test.transpose(0, 1)
    x_test = torch.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    x_test = x_test.to(device)

    label_test = y_test[st:ed]
    label_test = label_test.to(device)

    output_test = model(x_test, ftr_all, adj, X_train.shape[0] + st, X_train.shape[0] + ed)
    predict_test = torch.squeeze(output_test)
    y_pred += predict_test.gt(0.5)

y_pred = list(map(float, y_pred))
y_pred = np.asarray(y_pred)
y_pred = torch.FloatTensor(y_pred)

print('Transformer Results:')
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('Roc:', metrics.roc_auc_score(y_test, y_pred))